In [ ]:
import os
import ntpath
import cv2
import numpy as np
from PIL import Image, ImageChops, ImageDraw, ImageOps, ImageFilter, ImageStat, ImageEnhance 
#from skimage import io, color, exposure, transform
#import imutils
import argparse
import sys
import matplotlib
import matplotlib.pyplot as plt
import glob
import math

import shutil
from datetime import datetime
import random
#from blend_modes import blend_modes
import blend_modes

original = True # Whether we are using the original exposure_manipulation code or not

In [ ]:
# Return a list with paths of all files in the directory
def load_paths(directory):
    paths = []
    for filename in os.listdir(directory): # Retrieve names of files in directory
        # Concatenate filename with directory path, ignoring hidden files
        path = os.path.join(directory, filename)
        if not filename.startswith('.'):
            paths.append(path)
    return paths

In [ ]:
# Return a list with paths of all non-directory files in the directory
def load_files(directory):
    paths = []
    for filename in os.listdir(directory): # Retrieve names of files in directory
        # Concatenate filename with directory path, ignoring hidden files and directories
        path = os.path.join(directory, filename)
        if os.path.isfile(path) and not filename.startswith('.'):
            paths.append(path)
    return paths

In [ ]:
# Based on Alexandros Stergiou's "find_borders()"
# Returns an alpha channel that matches the white background
def create_alpha(img, alpha_channel):
    # Read and decode the image contents for pixel access
    pix = img.load()

    # Note PIL indexes [x,y] while OpenCV indexes [y,x]
    # alpha_channel must be indexed [y,x] to merge with OpenCV channels later

    min = 200
    width, height = img.size
    # Loop through each row of the image
    for y in range(0, height):
        # First loop left to right
        for x in range(0, width, 1):
            # Retrieve a tuple with RGB values for this pixel
            rgb = pix[x,y]
            # Make transparent if the pixel is white (or light enough)
            if rgb[0] >= min and rgb[1] >= min and rgb[2] >= min:
                alpha_channel[y,x] = 0
            # If pixel is not white then we've hit the sign so break out of loop
            else:
                break

        # Then loop backwards, right to left
        for x in range(width-1, -1, -1):
            rgb = pix[x,y]
            if rgb[0] >= min and rgb[1] >= min and rgb[2] >= min:
                alpha_channel[y,x] = 0
            else:
                break

    return alpha_channel

In [ ]:
# Based on Alexandros Stergiou's "manipulate_images()"
# Delete the white background from the original sign
def delete_background(image_path, output_dir):
    # Open the image using PIL (don't read contents yet)
    img = Image.open(image_path)
    img = img.convert('RGB')  # Does this have any effect?

    # Open the image again using OpenCV and split into its channels
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    channels = cv2.split(image)

    # Create a fully opaque alpha channel, same dimentions and dtype as the image
    # create_alpha() modifies it to make the white background transparent
    alpha_channel = np.ones(channels[0].shape, dtype=channels[0].dtype) * 255
    alpha_channel = create_alpha(img, alpha_channel)

    # Merge alpha channel into original image
    image_RGBA = cv2.merge((channels[0], channels[1], channels[2], alpha_channel))

    _,filename = ntpath.split(image_path)
    # Remove the extension and save as a png
    name,_ = filename.rsplit('.', 1)
    save_path = os.path.join(output_dir, name) + ".png"
    cv2.imwrite(save_path, image_RGBA)

    img.close()

In [ ]:
# Make white backgrounds transparent
input_path = "Original Sets/Images/"
output_path = "Traffic_Signs_Templates/2_Processed_Images/"
# Create the output directory if it doesn't exist already
if (not os.path.exists(output_path)):
    os.mkdir(output_path)

paths = load_files(input_path)
for path in paths:
    delete_background(path, output_path)

In [ ]:
# Resize the first image if it is larger than the second image
'''
def resize(img1, img2):
    # Retrieve dimentions of both images
    height1, width1, _ = img1.shape  # Discard channel
    height2, width2, _ = img2.shape

    # Resize if foreground is taller than background
    if height1 > height2:
        img1 = match_height(img1, height2)
        height1, width1, _ = img1.shape  # Re-retrieve dimentions
    # Or wider
    if width1 > width2:
        img1 = match_width(img1, width2)

    return img1, img2


def match_height(img, new_height):
    old_height, old_width, _ = img.shape  # Discard channel
    new_width = int(round( new_height / old_height * old_width ))
    img = cv2.resize(img, (new_height, new_width))
    return img


def match_width(img, new_width):
    old_width, old_height, _ = img.shape
    new_height = int(round( new_width / old_width * old_height ))
    img = cv2.resize(img, (new_width, new_height))
    return img
'''

In [ ]:
# Overlays foreground image on background image, keeping transparency
# x1, y1: top-left coordinate to place foreground
# Image overlay code credit to fireant:
# https://stackoverflow.com/questions/14063070/overlay-a-smaller-image-on-a-larger-image-python-opencv
def overlay(fg, bg, x1=-1, y1=-1):
    # If the background doesn't have an alpha channel, add one, but keep the entire image opaque
    if len(cv2.split(bg)) == 3:
        bg = cv2.cvtColor(bg, cv2.COLOR_RGB2RGBA)
        bg[:, :, 3] = 255
    # Make a copy of the background for making changes
    new_img = bg.copy()

    # Retrieve image dimentions
    height_FG, width_FG, _ = fg.shape  # Discard channel
    height_BG, width_BG, _ = bg.shape

    # If either of the coordinates were omitted, calculate start/end positions
    # using the difference in image size, centring foreground on background
    if x1 == -1 or y1 == -1:
        # Start coordinates 
        x1 = (width_BG - width_FG) // 2    # Floor division to truncate as
        y1 = (height_BG - height_FG) // 2  # coordinates don't need to be exact
    # End coordinates
    x2 = x1 + width_FG
    y2 = y1 + height_FG

    ### Start of code from fireant ###
    # Retrieve an array of alpha values from the foreground image
    # Divide by 255 to get values between 0.0 and 1.0
    alpha = fg[:,:,3] / 255.0
    beta = 1.0 - alpha

    # Loop over BGR channels (but not alpha)
    for ch in range(0, 3):
        new_img[y1:y2, x1:x2, ch] = (alpha * fg[:, :, ch] +
                                     beta * new_img[y1:y2, x1:x2, ch])
    ### End of code from fireant ###

    return new_img

In [ ]:
# Calculate the percentage of obscurity of graffiti
# calculates the ratio of opacity in first image compared to second image
def calc_ratio(fg, bg):
    fgPixels = count_pixels(fg)  # number of non-transparent pixels in graffiti
    bgPixels = count_pixels(bg)  # total number of pixels, dependent on sign shape
    ratio = fgPixels / bgPixels

    return ratio


# Return the number of non-transparent pixels in the imported image
def count_pixels(img):
    sum = 0  # Initialise to 0 in case there is no alpha channel
    split = cv2.split(img)
    if len(split) is 4:  # Only proceed if the image has an alpha channel
        alpha = split[3]
        # Loop through alpha channel
        # Divide alpha value by 255 to weight the transparency
        for ii in range(0, len(alpha)):
            for jj in range(0, len(alpha[0])):
                sum += alpha[ii][jj] / 255
    return sum

In [ ]:
# Draw some text to represent graffiti
# Returns the image and the ratio of oscurity
def graffiti_writing(img):
    # Create a new image the same dimentions as the image but completely transparent
    height,width,_ = img.shape
    grft = np.zeros((height, width, 4), dtype=np.uint8)
    # Retrieve the original alpha data to reapply later
    alphaData = cv2.split(img)[3]

    # Draw some graffiti onto the transparent image
    xx,yy = int(width * 0.1), int(height * 0.5)
    grft = write(grft, "GRFT", xx, yy, scale=3, color_bgr=(25,25,25))
    xx,yy = int(width * 0.3), int(height * 0.8)
    grft = write(grft, "DAMAGE", xx, yy, scale=2, color_bgr=(230,170,0))
    # Apply a Gaussian blur to smooth the edges
    k = (int(round( width/40 )) // 2) * 2 + 1  # Kernel size must be odd
    grft = cv2.GaussianBlur(grft, (k,k), 0)

    # Use the two separate images to calculate the ratio of obscurity
    temp = calc_ratio(grft, img)
    ratio = "{:0.3f}".format( temp )

    # Overlay the graffiti onto the sign
    img = overlay(grft, img)
    # Reapply the alpha data to cover up any graffiti that spilled over the sign
    img[:,:,3] = alphaData
    
    return img, ratio


def write(img, text, xx, yy, scale, color_bgr):
    fontFace = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX
    _,width,_ = img.shape
    fontScale = width / 400 * scale
    thickness = int(round( width / 120 * scale ))

    # Put one letter at a time, overlapping the letters slightly
    (wd,ht),_ = cv2.getTextSize(text, fontFace, fontScale, thickness)
    for letter in text:
        # Gnerate some randomness in transparency
        alpha = random.randint(240,255)
        color = color_bgr + (alpha,)
        # Use a random number to generate some range in letter size
        rand = random.uniform(0.7, 1.3)
        origin = ( xx, yy - int(round(ht*(1-rand)/2)) )  # Centre vertical position
        cv2.putText(img, letter, origin, fontFace, fontScale*rand, color, thickness, cv2.LINE_AA)
        # Calculate the horizontal starting position for the next letter, based on this character's size
        offset = int(round( wd / len(text) * 0.6 * rand ))  # 60% character width
        xx = xx + offset
    
    return img

In [ ]:
# Obscure a sign with scribbled graffiti
# Returns the image and the ratio of oscurity
def graffiti_scribble(img):
    # Create a new image the same dimentions as the image but completely transparent
    height,width,_ = img.shape
    blank = np.zeros((height, width, 4), dtype=np.uint8)
    # Retrieve the original alpha data to reapply after overlaying
    alphaData = cv2.split(img)[3]

    dmg = {}  # Dictionary to map the images to their obscurity percentages

    # Turning point coordinates, for a 1x1 image
    pts1 = ( (0.28,0.12), (0.10,0.44), (0.46,0.12), (0.14,0.68), (0.70,0.16),
             (0.30,0.84), (0.86,0.32), (0.54,0.88), (0.90,0.56), (0.72,0.88) )
    # Create a horizontal reflection of the points
    pts2 = ()
    for ii in range(len(pts1)):
        pts2 = pts2 + ( (1.0 - pts1[ii][0], pts1[ii][1]), )

    # Scribble in 4 different colours
    colors = ( (240,240,240), (230,170,0), (140,70,30), (0,0,0) )
    # Kernel size for Gaussian blur, which must be odd
    k = (int(round( width/15 )) // 2) * 2 + 1

    for color in colors:
        grft = blank.copy()
        grft = draw_lines(grft, pts1, color)
        # Apply a Gaussian blur to smooth the edges
        grft = cv2.GaussianBlur(grft, (k,k), 0)

        temp = overlay(grft, img)
        # Reapply the alpha data to cover up any graffiti that spilled over the sign
        temp[:,:,3] = alphaData

        ratio = "{:0.3f}".format( calc_ratio(grft, img) )
        dmg[ratio] = temp  # Add to dictionary

    # Overlay 2 scribbles to create a more obscured sign
    # Left facing
    grft1 = blank.copy()
    grft1 = draw_lines(grft1, pts1, colors[1])
    grft1 = cv2.GaussianBlur(grft1, (k,k), 0)
    # Right facing
    grft2 = blank.copy()
    grft2 = draw_lines(grft2, pts2, colors[2])
    grft2 = cv2.GaussianBlur(grft2, (k,k), 0)
    # Merge the 2 scribbles
    # Modify grft2's alpha channel to the union of both scribbles before overlaying 
    alpha_combined = cv2.bitwise_or(cv2.split(grft1)[3], cv2.split(grft2)[3])
    grft2[:,:,3] = alpha_combined
    grft = overlay(grft1, grft2)

    temp = overlay(grft, img)  # Overlay with the sign
    temp[:,:,3] = alphaData  # Reapply alpha data

    ratio = "{:0.3f}".format( calc_ratio(grft, img) )
    dmg[ratio] = temp  # Add to dictionary

    return dmg


def draw_lines(img, pts, color_bgr):
    height,width,_ = img.shape
    for ii in range(len(pts)-1):
        # Generate some randomness in line thickness and colour transparency
        thickness = int(round( random.uniform(0.8, 1.2) * width/15 ))
        alpha = random.randint(245, 255)
        color = color_bgr + (alpha,)
        # Multiply by width or height (and round) to get coordinates for drawing the lines
        pt1 = ( int(round(width * pts[ ii ][0])), int(round(height * pts[ ii ][1])) )
        pt2 = ( int(round(width * pts[ii+1][0])), int(round(height * pts[ii+1][1])) )
        cv2.line(img, pt1, pt2, color, thickness, cv2.LINE_AA)
    
    return img

In [ ]:
# Apply perspective warp to tilt images and combine to produce bent signs
# Returns a dictionary with the bent signs mapped to the filenames to save as
def bend_vertical(img_path):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    ht,wd,_ = img.shape  # Retrieve image dimentions

    dmg = {}  # Dictionary to map the images to filenames

    # TILT 1
    pt = wd // 24
    xx = pt * 3
    yy = pt // 2
    # Right             Top-left Top-middle  Bottom-left Bottom-middle
    src = np.float32( [ [pt,pt], [wd//2,pt], [pt,ht-pt], [wd//2,ht-pt] ] )
    dst = np.float32( [ [xx,yy], [wd//2,pt], [xx,ht-yy], [wd//2,ht-pt] ] )
    matrix = cv2.getPerspectiveTransform(src, dst)
    right = cv2.warpPerspective(img, matrix, (wd,ht))
    # Left              Top-middle  Top-right   Bottom-middle  Bottom-right
    src = np.float32( [ [wd//2,pt], [wd-pt,pt], [wd//2,ht-pt], [wd-pt,ht-pt] ] )
    dst = np.float32( [ [wd//2,pt], [wd-xx,yy], [wd//2,ht-pt], [wd-xx,ht-yy] ] )
    matrix = cv2.getPerspectiveTransform(src, dst)
    left = cv2.warpPerspective(img, matrix, (wd,ht))
    
    # Combine the left and right tilt with the original forward-facing image
    dmg["0_40"] = combine(img, right)
    dmg["40_0"]  = combine(left, img)
    # Combine the left and right tilt
    dmg["40_40"] = combine(left, right)

    # TILT 2
    pt = wd // 12
    xx = pt * 3
    yy = pt // 2
    # Right             Top-left Top-middle  Bottom-left Bottom-middle
    src = np.float32( [ [pt,pt], [wd//2,pt], [pt,ht-pt], [wd//2,ht-pt] ] )
    dst = np.float32( [ [xx,yy], [wd//2,pt], [xx,ht-yy], [wd//2,ht-pt] ] )
    matrix = cv2.getPerspectiveTransform(src, dst)
    right = cv2.warpPerspective(img, matrix, (wd,ht))
    # Left              Top-middle  Top-right   Bottom-middle  Bottom-right
    src = np.float32( [ [wd//2,pt], [wd-pt,pt], [wd//2,ht-pt], [wd-pt,ht-pt] ] )
    dst = np.float32( [ [wd//2,pt], [wd-xx,yy], [wd//2,ht-pt], [wd-xx,ht-yy] ] )
    matrix = cv2.getPerspectiveTransform(src, dst)
    left = cv2.warpPerspective(img, matrix, (wd,ht))
    
    # Combine the left and right tilt with the original forward-facing image
    dmg["0_60"] = combine(img, right)
    dmg["60_0"]  = combine(left, img)
    # Combine the left and right tilt
    dmg["60_60"] = combine(left, right)

    return dmg


# Combines the left half of img1 with right half of img2 and returns the result
def combine(img1, img2):
    _,wd,_ = img1.shape
    result = img1.copy()

    # Save the alpha data (to replace later), as convertScaleAbs() will affect the transparency
    alphaData = cv2.split(result)[3]

    # Darken the entire image of the copy
    alpha = 1   # No change to contrast
    beta = -20  # Decrease brightness
    cv2.convertScaleAbs(result, result, alpha, beta)
    # Replace the alpha data
    result[:,:,3] = alphaData

    # Copy over the right half of img2 onto the darkened image
    result[:,wd//2:wd] = img2[:,wd//2:wd]

    return result

In [ ]:
# Used for SGTSD
damage_types = ["_ORIGINAL", "_QUADRANT_4", "_BOTTOM_HOLE", "_BULLET", "_YELLOW", \
                "_GRAFFITI", "_BEND", "_GREY"]

def damage_images(image_path, output_dir):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    img = img.astype('uint8')
    height,width,ch = img.shape
    centre_x = int(width/2)
    centre_y = int(height/2)
    
    # Create a directory for this sign number
    _,tail = ntpath.split(image_path)  # Retrieve the filename of the image
    title,_ = tail.split('.')          # Discard extension
    output_path = os.path.join(output_dir, title)
    # Create the output directory if it doesn't exit already
    if (not os.path.exists(output_path)):
        os.makedirs(output_path)

    # Create a mask
    sign = img.copy() 
    alpha_ch = sign[:, :, 3] # Extract the alpha channel from sign
    _,mask = cv2.threshold(alpha_ch, 5, 255, cv2.THRESH_BINARY) # Remove gradual transparency

    
    # ORIGINAL UNDAMAGED
    cv2.imwrite(os.path.join(output_path, title + ".png"), img)
    

    # TOP-LEFT QUADRANT
    top_left = mask.copy()
    cv2.rectangle(top_left, (0,0), (centre_x,centre_y), (0,0,0), -1)
    dmg1 = cv2.bitwise_and(img, img, mask=top_left)
    
    cv2.imwrite(os.path.join(output_path, title + damage_types[1] + ".png"), dmg1)
    

    # BOTTOM HOLE
    bottom_hole = mask.copy()
    cv2.circle(bottom_hole, (centre_x,height), int(height/2), (0,0,0), -1)
    dmg2 = cv2.bitwise_and(img, img, mask=bottom_hole)
    
    cv2.imwrite(os.path.join(output_path, title + damage_types[2] + ".png"), dmg2)
    

    # RANDOMISED BULLET HOLES
    bullet_holes = mask.copy()
    painted = img.copy() # Another copy of original sign to draw 'flaking paint' from holes on
    numHoles = random.randint(7, 30)
    for x in range(numHoles):
        size = random.randint(2, 12) # Random hole size
        h_x = random.randint(0, width) # Random hole position
        h_y = random.randint(0, height)
        c = random.randint(0, 150) # How black/grey the 'hole' is if it didn't penetrate
        s = random.uniform(1.6, 2.2) # Random annulus size

        C = 200 # Colour of damaged 'paint' outer annulus
        cv2.circle(painted, (h_x, h_y), int(size * s), (C,C,C,255), -1) # Hole annulus to represent damaged 'paint'
        # Did the bullet penetrate through the sign?
        if (size < 6):
            cv2.circle(painted, (h_x, h_y), size, (c,c,c,255), -1)
        # If not, grey out rather than make transparent
        else:
            cv2.circle(bullet_holes, (h_x, h_y), size, (0,0,0), -1)
    dmg3 = cv2.bitwise_and(painted, painted, mask=bullet_holes)
    
    cv2.imwrite(os.path.join(output_path, title + damage_types[3] + ".png"), dmg3)
    

    # TINTED YELLOW
    yellow = np.zeros((height,width,ch), np.uint8)
    yellow[:,:] = (0,210,210,255)
    dmg4 = cv2.bitwise_and(img, yellow)
    
    cv2.imwrite(os.path.join(output_path, title + damage_types[4] + ".png"), dmg4)
    

    # GRAFFITI
    dmg5,ratio = graffiti_writing(img)
    cv2.imwrite(os.path.join(output_path, title + damage_types[5] + "_" + ratio + ".png"), dmg5)
    
    dmg5 = graffiti_scribble(img)
    for ratio,dmg in dmg5.items():
        cv2.imwrite(os.path.join(output_path, title + damage_types[5] + "_" + ratio + ".png"), dmg)
    

    # BEND
    dmg6 = bend_vertical(image_path)
    for detail,dmg in dmg6.items():
        cv2.imwrite(os.path.join(output_path, title + damage_types[6] + "_" + detail + ".png"), dmg)
    

    # GREY
    dmg7 = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
    _,dmg7 = cv2.threshold(dmg7, 200, 255, cv2.THRESH_BINARY)
    cv2.convertScaleAbs(dmg7, dmg7, alpha=1, beta=200)
    dmg7 = cv2.cvtColor(dmg7, cv2.COLOR_GRAY2BGRA)
    dmg7[:,:,3] = alpha_ch
    cv2.imwrite(os.path.join(output_path, title + damage_types[7] + ".png"), dmg7)

    # CRACKS (thin crack lines across the sign?)

In [ ]:
# Create damaged sign templates based on images with backgrounds removed
input_path = "Traffic_Signs_Templates/2_Processed_Images/"
output_dir = "Traffic_Signs_Templates/3_Damaged_Images/"

for image_path in load_files(input_path):
    damage_images(image_path, output_dir)

In [ ]:
# Add a pole to the imported sign
def add_pole(filename, color):
    img = cv.imread(filename, cv.IMREAD_UNCHANGED)
    # Retrieve image dimentions and calculate the new height
    height, width, _ = img.shape  # Discard channel
    new_height = height * 3

    # Create a new blank image with dtype=uint8 (to hold numbers 0-255)
    # Initialise values to 0 so that the extended image is fully transparent
    newImage = np.zeros((new_height, width, 4), dtype=np.uint8)
    # Copy over the original image onto the top portion
    newImage[0:height, :] = img

    # Calculate coordinates
    midpt = width // 2
    offset = width // 40
    x1, x2 = midpt - offset, midpt + offset
    y1, y2 = height, new_height
    # Draw the pole
    cv.rectangle(newImage, (x1, y1), (x2, y2), color, cv.FILLED)

    # Colour any transparent pixels between the sign and the rectangle
    lim = 50  # Max alpha value for the pixel to be considered "transparent"
    margin = int(round( height * 0.9 ))  # Loop over bottom 10% of the sign
    for y in range(margin, height):
        for x in range(x1, x2+1):  # Loop to x2 inclusive
            if newImage[y,x,3] < lim:
                newImage[y,x] = color

    return newImage

In [ ]:
# Add a pole and place on cityscapes

In [ ]:
def img_transform(image_path, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    height,width,_ = img.shape

    dst = []
    #0 FORWARD FACING
    dst.append( img )

    #1 EAST FACING
    pts1 = np.float32( [[width/10,height/10], [width/2,height/10], [width/10,height/2]] )
    pts2 = np.float32( [[width/5,height/5], [width/2,height/8], [width/5,height/1.8]] )
    M = cv2.getAffineTransform(pts1,pts2)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #2 NORTH-WEST FACING
    pts3 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts4 = np.float32( [[width*9/10,height/5], [width/2,height/8], [width*9/10,height/1.8]] )
    M = cv2.getAffineTransform(pts3,pts4)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #3 LEFT TILTED FORWARD FACING
    pts5 = np.float32( [[width/10,height/10], [width/2,height/10], [width/10,height/2]] )
    pts6 = np.float32( [[width/12,height/6], [width/2.1,height/8], [width/10,height/1.8]] )
    M = cv2.getAffineTransform(pts5,pts6)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #4 RIGHT TILTED FORWARD FACING
    pts7 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts8 = np.float32( [[width*10/12,height/6], [width/2.2,height/8], [width*8.4/10,height/1.8]] )
    M = cv2.getAffineTransform(pts7,pts8)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #5 WEST FACING
    pts9  = np.float32( [[width/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts10 = np.float32( [[width/9.95,height/10], [width/2.05,height/9.95], [width*9/10,height/2.05]] )
    M = cv2.getAffineTransform(pts9,pts10)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #6 RIGHT TILTED FORWARD FACING
    pts11 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts12 = np.float32( [[width*9/10,height/10], [width/2,height/9], [width*8.95/10,height/2.05]] )
    M = cv2.getAffineTransform(pts11,pts12)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #7 FORWARD FACING W/ DISTORTION
    pts13 = np.float32( [[width/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts14 = np.float32( [[width/9.8,height/9.8], [width/2,height/9.8], [width*8.8/10,height/2.05]] )
    M = cv2.getAffineTransform(pts13,pts14)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #8 FORWARD FACING W/ DISTORTION 2
    pts15 = np.float32( [[width/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts16 = np.float32( [[width/11,height/10], [width/2.1,height/10], [width*8.5/10,height/1.95]] )
    M = cv2.getAffineTransform(pts15,pts16)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #9 FORWARD FACING W/ DISTORTION 3
    pts17 = np.float32( [[width/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts18 = np.float32( [[width/11,height/11], [width/2.1,height/10], [width*10/11,height/1.95]] )
    M = cv2.getAffineTransform(pts17,pts18)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #10 FORWARD FACING W/ DISTORTION 4
    pts19 = np.float32( [[width*9.5/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts20 = np.float32( [[width*9.35/10,height/9.99], [width/2.05,height/9.95], [width*9.05/10,height/2.03]] )
    M = cv2.getAffineTransform(pts19,pts20)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #11 FORWARD FACING W/ DISTORTION 5
    pts21 = np.float32( [[width*9.5/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts22 = np.float32( [[width*9.65/10,height/9.95], [width/1.95,height/9.95], [width*9.1/10,height/2.02]] )
    M = cv2.getAffineTransform(pts21,pts22)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #12 FORWARD FACING W/ DISTORTION 6
    pts23 = np.float32( [[width*9.25/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts24 = np.float32( [[width*9.55/10,height/9.85], [width/1.9,height/10], [width*9.3/10,height/2.04]] )
    M = cv2.getAffineTransform(pts23,pts24)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #13 SHRINK 1
    pts25 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts26 = np.float32( [[width*8/10,height/10], [width*1.34/3,height/10.5], [width*8.24/10,height/2.5]] )
    M = cv2.getAffineTransform(pts25,pts26)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #14 SHRINK 2
    pts27 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts28 = np.float32( [[width*8.5/10,height*3.1/10], [width/2,height*3/10], [width*8.44/10,height*1.55/2.5]] )
    M = cv2.getAffineTransform(pts27,pts28)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #15 FORWARD FACING W/ DISTORTION 7
    pts29 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts30 = np.float32( [[width*8.85/10,height/9.3], [width/1.9,height/10.5], [width*8.8/10,height/2.11]] )
    M = cv2.getAffineTransform(pts29,pts30)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #16 FORWARD FACING W/ DISTORTION 8
    pts31 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts32 = np.float32( [[width*8.75/10,height/9.1], [width/1.95,height/8], [width*8.5/10,height/2.05]] )
    M = cv2.getAffineTransform(pts31,pts32)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #17 FORWARD FACING W/ DISTORTION 9
    pts33 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts34 = np.float32( [[width*8.75/10,height/9.1], [width/1.95,height/9], [width*8.5/10,height/2.2]] )
    M = cv2.getAffineTransform(pts33,pts34)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #18 FORWARD FACING W/ DISTORTION 10
    pts35 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts36 = np.float32( [[width*8.75/10,height/8], [width/1.95,height/8], [width*8.75/10,height/2]] )
    M = cv2.getAffineTransform(pts35,pts36)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #19 FORWARD FACING W/ DISTORTION 11
    pts37 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts38 = np.float32( [[width*8.8/10,height/7], [width/1.95,height/7], [width*8.8/10,height/2]] )
    M = cv2.getAffineTransform(pts37,pts38)
    dst.append( cv2.warpAffine(img,M,(width,height)) )

    # Retrieve the filename to save as
    _,tail = ntpath.split(image_path)  # Filename of the image, parent directories removed
    title,_ = tail.rsplit('.', 1)      # Discard extension
    
    # Save the transformed images
    for ii in range(1, len(dst)):
        save_path = os.path.join(output_path, title)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        cv2.imwrite(os.path.join(save_path, str(ii) + ".png"), dst[ii])
#    plt.show()

In [ ]:
#input_path = "Traffic_Signs_Templates/1_Images/"
#input_path = "Traffic_Signs_Templates/2_Processed_Images/"
input_path = "Traffic_Signs_Templates/3_Damaged_Images/"
output_dir = "Traffic_Signs_Templates/4_Transformed_Images/"

# Loop through each folder in the input directory
for folder_path in load_paths(input_path):
    # Retrieve the sign number to create a directory
    _,number = os.path.split(folder_path)
    save_dir = os.path.join(output_dir, number)
    
    for image_path in load_files(folder_path):
        img_transform(image_path, save_dir)

In [ ]:
def to_png(directory):
    for files in load_paths(directory):
        title,extension = files.split('.')
        img = Image.open(files).convert('RGBA')
        if (not extension == "png"):
            os.remove(files)
        img.save(title+".png")

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#to_png("Google_search_backgrounds/UK_urban")
to_png("Google_search_backgrounds/UK_rural")

In [ ]:
def find_image_exposure(paths, channels):
    exposures = []
    for image_path in paths:
        img = Image.open(image_path)
        im = Image.open(image_path).convert('LA')
        
        stat = ImageStat.Stat(im)
        
        #Average pixel brighness
        avg = stat.mean[0]
        
        #RMS pixel brighness
        rms = stat.rms[0]
        
        stat2 = ImageStat.Stat(img)
        
        #Consider the number of channels
        #background may have RGB while traffic sign has RGBA
        if (channels==3):
            #Average pixels preceived brightness
            r,g,b = stat2.mean
            avg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

            #RMS pixels perceived brightness
            r,g,b = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2)) 

            l = [image_path,avg,rms,avg_perceived,rms_perceived]
            exposures.append(l)
        else:
            #Average pixels preceived brightness
            r,g,b,a = stat2.mean
            avg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

            #RMS pixels perceived brightness
            r,g,b,a = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2)) 

            l = [image_path,avg,rms,avg_perceived,rms_perceived]
            exposures.append(l)

    return exposures     

In [ ]:
def exposure_manipulation(signs_paths, backgrounds_paths):
    background_exposures = find_image_exposure(background_paths,4)
    signs_exposures = find_image_exposure(signs_paths,4)
    
    for i in range(0,len(background_paths)):
        print("Processed: " + str(float(i) / float(len(background_paths)) * 100) + " %")
        
        img = Image.open(background_exposures[i][0])

        for sign_path in signs_paths:        
            dirc,sub,el = background_exposures[i][0].split('/')
            title,extension = el.split('.')

            parent_dir,sub_dir,folder,folder2,element = sign_path.split('/')
            head,tail = element.split('.')

            
            ###   ORIGINAL EXPOSURE IMPLEMENTATION   ###
            brightness_avrg = 1.0
            brightness_rms = 1.0
            brightness_avrg_perceived = 1.0
            brightness_rms_perceived = 1.0
            brightness_avrg2 = 1.0
            brightness_rms2 = 1.0

            # abs(desired_brightness - actual_brightness) / abs(brightness_float_value) = ratio
            avrg_ratio = 11.0159464507

            rms_ratio = 8.30320014372

            percieved_avrg_ratio = 3.85546373056

            percieved_rms_ratio = 35.6344530649

            avrg2_ratio = 1.20354549572

            rms2_ratio = 40.1209106864

            peak = Image.open(sign_path).convert('LA')
            peak2 = Image.open(sign_path).convert('RGBA')

            stat = ImageStat.Stat(peak)
            avrg = stat.mean[0]
            rms = stat.rms[0]

            
            #IMAGE MANIPULATION MAIN CODE STARTS

            #MINIMISE MARGIN BASED ON AVERAGE FOR TWO CHANNEL BRIGNESS VARIATION
            margin = abs(avrg-float(background_exposures[i][1]))
            
            brightness_avrg = margin/avrg_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright = enhancer.enhance(brightness_avrg)
            stat = ImageStat.Stat(avrg_bright)
            avrg = stat.mean[0]

            
            #MINIMISE MARGIN BASED ON ROOT MEAN SQUARE FOR TWO CHANNEL BRIGNESS VARIATION
            margin = abs(rms-float(background_exposures[i][2]))

            brightness_rms = margin/rms_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright = enhancer.enhance(brightness_rms)
            stat = ImageStat.Stat(rms_bright)
            rms = stat.rms[0]

            
            #MINIMISE MARGIN BASED ON AVERAGE FOR RGBA ("PERCEIVED BRIGHNESS")
            #REFERENCE FOR ALGORITHM USED: http://alienryderflex.com/hsp.html
            stat2 = ImageStat.Stat(peak2)
            r,g,b,a = stat2.mean
            avrg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))
            margin = abs(avrg_perceived-float(background_exposures[i][3]))
            
            brightness_avrg_perceived = margin/percieved_avrg_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright_perceived = enhancer.enhance(brightness_avrg_perceived)
            stat2 = ImageStat.Stat(avrg_bright_perceived)
            r,g,b,a = stat2.mean
            avrg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))        


            #MINIMISE MARGIN BASED ON RMS FOR RGBA ("PERCEIVED BRIGHNESS")
            #REFERENCE FOR ALGORITHM USED: http://alienryderflex.com/hsp.html
            r,g,b,a = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

            margin = abs(rms_perceived-float(background_exposures[i][4]))

            brightness_rms_perceived = margin/percieved_rms_ratio 

            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright_perceived = enhancer.enhance(brightness_rms_perceived)
            stat2 = ImageStat.Stat(rms_bright_perceived)
            r,g,b,a = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))        

            
            stat3 = ImageStat.Stat(peak2)
            avrg2 = stat3.mean[0]
            rms2 = stat3.rms[0]

            """
            #FUSION OF THE TWO AVERAGING METHODS
            margin = abs(avrg2-float(background_exposures[i][1]))
            brightness_avrg2 = margin/avrg2_ratio 
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright2 = enhancer.enhance(brightness_avrg2)
            stat3 = ImageStat.Stat(avrg_bright2)
            avrg2 = stat3.mean[0]       
            """
            
            
            """
            #FUSION OF THE TWO RMS METHODS
            margin = abs(rms2-float(background_exposures[i][2]))
            brightness_rms2 = margin/rms2_ratio 
            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright2 = enhancer.enhance(brightness_rms2)
            stat3 = ImageStat.Stat(rms_bright2)
            rms2 = stat3.rms[0]
            """
            
            avrg_bright = avrg_bright.resize((150,150), Image.ANTIALIAS)
            rms_bright = rms_bright.resize((150,150), Image.ANTIALIAS)
            avrg_bright_perceived = avrg_bright_perceived.resize((150,150), Image.ANTIALIAS)
            rms_bright_perceived = rms_bright_perceived.resize((150,150), Image.ANTIALIAS)
            #avrg_bright2 = avrg_bright2.resize((150,150), Image.ANTIALIAS)
            #rms_bright2 = rms_bright2.resize((150,150), Image.ANTIALIAS)

            
            exp_dir = "Traffic_Signs_Exposure_Manipulation/"
            avrg_bright.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_AVERAGE."+tail)
            rms_bright.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_RMS."+tail)
            avrg_bright_perceived.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_AVERAGE_PERCEIVED."+tail)
            rms_bright_perceived.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_RMS_PERCEIVED."+tail)
            #avrg_bright2.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_AVERAGE2."+tail)
            #rms_bright2.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_RMS2."+tail)
    
    print("Processed: " + str(100) + " %")
    print("Process was successful")

In [ ]:
def fade_manipulation(signs_paths, backgrounds_paths):
    background_exposures = find_image_exposure(background_paths,4)
    signs_exposures = find_image_exposure(signs_paths,4)
    
    print("Processed: 0.0 %")
    ii = 0
    prev = 0
    for sign_path in signs_paths:
        progress = float(ii) / float(len(signs_paths)) * 100
        if progress >= prev + 5: #Prevent spamming of progress prints
            prev = prev + 5
            print("Processed: " + str(progress) + " %")

        dirc,sub,el = background_exposures[0][0].split('/')
        title,extension = el.split('.')

        parent_dir,sub_dir,folder,folder2,element = sign_path.split('/')
        head,tail = element.split('.')

        img = cv2.imread(sign_path, cv2.IMREAD_UNCHANGED)


        ###   GRADUAL FADE IMPLEMENTATION   ###
        #Retrieve alpha data from original image
        splitImg = cv2.split(img)
        if len(splitImg) is 4:
            alphaData = splitImg[3]

        for jj in range(0,5):
            dmg6 = img.copy()
            alpha = 1 - (jj * 0.19)
            beta = (jj + 1) * 40
            if jj > 0:
                cv2.convertScaleAbs(img, dmg6, alpha, beta) #Scale the contrast and brightness
                dmg6[:, :, 3] = alphaData

            dmg6 = cv2.resize(dmg6, (150,150))
            fad_dir = "Traffic_Signs_Fade_Manipulation/"
            cv2.imwrite(fad_dir+"SIGN_"+folder+"/"+folder2+"/"+head+"_FADE-"+str(jj)+"."+tail, dmg6)
        ii = ii + 1
    
    
    print("Processed: " + str(100) + " %")
    print("Process was successful")

In [ ]:
bg_dir = "Google_search_backgrounds"

for dirs in load_paths(bg_dir):
    initial, subd = dirs.split('/')

    if original is True:
        for background in load_paths(dirs):
            initial, subd, element = background.split('/')
            title, extension = element.split('.')

            for signp in load_paths("Traffic_Signs_Templates/4_Transformed_Images"):
                for sign in load_paths(signp):
                    d,s,f,e = sign.split('/') #Eg. s = 4_Transformed_Images, f = 9, e = 9_BOTTOM_HOLE

                    exp_dir = "Traffic_Signs_Exposure_Manipulation/"
                    if (not os.path.exists(exp_dir + subd + "/" + title + "/SIGN_" + f + "/" + e)):
                        os.makedirs(exp_dir + subd + "/" + title + "/SIGN_" + f + "/" + e)
    else:
        for signp in load_paths("Traffic_Signs_Templates/4_Transformed_Images"):
            for sign in load_paths(signp):
                d,s,f,e = sign.split('/')

                fade_dir = "Traffic_Signs_Fade_Manipulation/"
                if (not os.path.exists(fade_dir + "SIGN_" + f + "/" + e)):
                    os.makedirs(fade_dir + "SIGN_" + f + "/" + e)

signs_paths = []
for p in load_paths("Traffic_Signs_Templates/4_Transformed_Images"):
    for d in load_paths(p):
        signs_paths = signs_paths + load_paths(d)

background_paths = load_paths("Google_search_backgrounds/UK_rural")
if original is True:
    exposure_manipulation(signs_paths, background_paths)
else:
    fade_manipulation(signs_paths, background_paths)

In [ ]:
#background_paths = load_paths("Google_search_backgrounds/UK_urban") #Don't do, takes too long to do both
#exposure_manipulation(signs_paths,background_paths)

In [ ]:
def avrg_pixel_rgb(image,chanels):
    stat = ImageStat.Stat(image)
    if (chanels == 4):
        r,g,b,a = stat.rms
    else:
        r,g,b = stat.rms
    
    return [r,g,b]

In [ ]:
def find_bw_images(directory):
    images = []
    for signs in load_paths(directory):
        img = Image.open(signs).convert('RGBA')
        rgb = avrg_pixel_rgb(img,4)
        rg = abs(rgb[0]-rgb[1])
        rb = abs(rgb[0]-rgb[2])
        gb = abs(rgb[1]-rgb[2])
        
        temp = signs.split('/')
        head,tail = temp[-1].split('.')
                
        if (rg<=1 and rb<=1 and gb<=1):
            images.append(head)
    return images

In [ ]:
def find_useful_signs(directory): #Removes bad signs, such as those which are all white or all black
    bw_images = find_bw_images("Traffic_Signs_Templates/3_Damaged_Images")
    for background_dir in load_paths(directory):
        for signs in load_paths(background_dir):
            for dmgs in load_paths(signs):
                temp = []
                for imgs in load_paths(dmgs):
                    temp.append(imgs)
                exposures = find_image_exposure(temp,4)
                
                i = 0
                for images in load_paths(dmgs):
                    #Find brightness
                    img = Image.open(images).convert('RGBA')

                    rgb = avrg_pixel_rgb(img,4)
                    rg = abs(rgb[0]-rgb[1])
                    rb = abs(rgb[0]-rgb[2])
                    gb = abs(rgb[1]-rgb[2])

                    is_bw = False

                    for s in bw_images:
                        if s in exposures[i][0]:
                            is_bw = True

                    if (rg<=16 and rb<=16 and gb<=16):
                        if (not is_bw):
                            os.remove(images)
                        #Threshold values for black and white images
                        elif (rgb[0]<70 and rgb[1]<70 and rgb[2]<70):
                            os.remove(images)
                        elif (rgb[0]>155 and rgb[1]>155 and rgb[2]>155):
                            os.remove(images)

                    elif (not is_bw):
                        #Delete light blue images
                        if(rgb[2]>rgb[0] and rgb[2]>=rgb[1]):
                            if (gb<=10):
                                os.remove(images)
                    i = i+1

In [ ]:
if original is True:
    directory = "Traffic_Signs_Exposure_Manipulation/UK_rural"
    find_useful_signs(directory)

In [ ]:
# if original is True:
#     directory= "Traffic_Signs_Exposure_Manipulation/UK_urban"
#     find_useful_signs(directory)

In [ ]:
def insert_poisson_noise (image):
    vals = len(np.unique(image))
    vals = 2.05 ** np.ceil(np.log2(vals))
    noisy = np.random.poisson(image * vals) / float(vals)
    return noisy

In [ ]:
def insert_Gaussian_noise (image):
    row,col,ch= image.shape
    mean = 0
    var = 0.5
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy

In [ ]:
def insert_speckle_noise (image):
    row,col,ch = image.shape
    gauss = np.random.randn(row,col,ch)
    gauss = gauss.reshape(row,col,ch)        
    noisy = image + image * gauss
    return noisy

In [ ]:
def random_noise_method (image):
    """
    i = random.randint(1, 3)
    if (i == 1):
        return insert_poisson_noise(image)
    elif (i==2):
        return insert_Gaussian_noise(image)
    else:
        return insert_speckle_noise(image)
    """
    image.setflags(write=1)
    #Add noise in every pixel w/ random probability 0.4
    for im in image:
        px = 0
        for pixel in im:
            apply_noise = random.randint(0,100)
            #if random probability
            if apply_noise > 40:
                #RGB values
                R = pixel[0]
                G = pixel[1]
                B = pixel[2]
                A = pixel[3]
                #find current relative lumination for brighness
                #based on: https://en.wikipedia.org/wiki/Relative_luminance
                relative_lumination = 0.2126*R + 0.7152*G + 0.0722*B
                #find differences between RGB values     
                R_to_G = float(R)/float(G)
                RG = False
                if (R_to_G >= 1): RG=True
                R_to_B = float(R)/float(B)
                RB = False
                if (R_to_B >= 1): RB=True
                G_to_B = float(G)/float(B)
                GB = False
                if (G_to_B >= 1): GB=True
                equal = False
                if (R==G==B):equal==True

                #In order to determine the margin in which the new brighness
                #should be within, the upper and lower limits need to be foun
                #The Relative luminance in colorimetric spaces has normilised
                #values between 0 and 255
                upper_limit = 255
                lower_limit = 0
                if (relative_lumination + 40 < 255):
                    upper_limit = relative_lumination + 40
                if (relative_lumination - 40 > 0):
                    lower_limit = relative_lumination - 40

                #Compute new brighness value
                new_lumination = random.randint(int(lower_limit),int(upper_limit))

                #find the three possible solutions that satisfy
                #->The new lumination chosen based on the Relative luminance equation
                #->The precentages computed between every RGB value

                solutions = []

                for r in range(1,255):
                    for g in range(1,255):
                        for b in range(1,255):
                            r_to_g = float(r)/float(g)
                            rg = False
                            if (r_to_g >= 1): rg=True
                            r_to_b = float(r)/float(b)
                            rb = False
                            if (r_to_b >= 1): rb=True
                            g_to_b = float(g)/float(b)
                            gb = False
                            if (g_to_b >= 1): gb=True
                            e = False
                            if(r==g==b):
                                e=True
                            if (0.2126*r + 0.7152*g + 0.0722*b == 100) and rg==RG and rb==RB and gb==GB and e==equal:
                                solutions.append([r,g,b])

                #Find the solution that precentage wise is closer to the original
                #difference between the values
                percentages = []

                for solution in solutions:
                    r = solution[0]
                    g = solution[1]
                    b = solution[2]
                    percentages.append((float(r)/float(g))+(float(r)/float(b))+(float(g)/float(b)))

                i = 0
                pos = 0
                best = percentages[0]
                for p in percentages[1:]:
                    if p < best:
                        pos = i
                    i = i +1

                #Assign new pixel values
                im[px] = [solutions[pos][0],solutions[pos][1],solutions[pos][2],A]
            px = px+1
            
    return image

In [ ]:
### ORIGINAL ###
# def new_data(image_dir,bg_dir): #Blends synthetic signs with backgrounds
#     # Import background image
#     background_img_raw = Image.open(bg_dir).convert('RGBA')  
#     background_img_raw = background_img_raw.resize((150,150), Image.ANTIALIAS)
#     background_img = np.array(background_img_raw)  
#     background_img_float = background_img.astype(float)  

#     # Import foreground image
#     foreground_img_raw = Image.open(image_dir)  
#     foreground_img = np.array(foreground_img_raw)  
#     foreground_img_float = foreground_img.astype(float)  

#     # Blend images
#     opacity = 1  
#     blended_img_float = blend_modes.grain_merge(background_img_float, foreground_img_float, opacity)

#     # Convert blended image back into PIL image
#     blended_img = np.uint8(blended_img_float)
#     blended_img_raw = Image.fromarray(blended_img)  
    
#     foreground_img_raw = foreground_img_raw.resize((149,149), Image.ANTIALIAS)
#     blended_img_raw.paste(foreground_img_raw, (0, 0), foreground_img_raw)
#     blended_img_raw = blended_img_raw.resize((48,48), Image.ANTIALIAS)
    
#     #temp = np.uint8(blended_img_raw)
#     #temp = random_noise_method(temp)
    
#     #blended_img_raw = Image.fromarray(np.uint8(temp)) 
    
#     return blended_img_raw

### FULL BACKGROUND ###
def new_data(image_dir,bg_dir): #Blends synthetic signs with backgrounds
    bg = cv2.imread(bg_dir, cv2.IMREAD_UNCHANGED)
    fg = cv2.imread(image_dir, cv2.IMREAD_UNCHANGED)
    return overlay(fg, bg)

In [ ]:
#Creating the required directories in SGTSD
directory = 'SGTSD/Images'
if (not os.path.exists("SGTSD/Images")):
    #Numbered Version
    for sign in load_paths("Traffic_Signs_Templates/1_Images"): #Folders for each sign type
        head, tail = sign.split('.')
        name = head.split('/')
        os.makedirs("SGTSD/Images/" + name[-1])
        j = 0
        for dmg in range(len(damage_types)): #Folders for each damage type
            os.makedirs("SGTSD/Images/" + name[-1] + "/" + name[-1] + "_" + str(j))
            j = j + 1
    
    #Named Version
#     for sign in load_paths("Traffic_Signs_Templates/1_Images"): #Folders for each sign type
#         head, tail = sign.split('.')
#         name = head.split('/')
#         os.makedirs("SGTSD/Images/" + name[-1])
#         for dmg in load_paths("Traffic_Signs_Templates/3_Damaged_Images"): #Folders for each damage type
#             headD,tailD = dmg.split('.')
#             nameD = headD.split('/')
#             os.makedirs("SGTSD/Images/" + name[-1] + "/" + nameD[-1])

In [ ]:
#Creating a README file
content = '''
-----------------------------------------------
|                     -*-                     |
|Synthetically Generated Traffic Sign Dataset |
|                     -*-                     |
-----------------------------------------------

This directory contains the training set for
The Convolutional Neural Network (CNN)
Used in this project

However, it can be used for any classifier
desired by the person using the code and
additionally, it is not limited to a specific
traffic sign templates.
 

----------------------------------------------
Content
----------------------------------------------

The number of example is based on the number:
->of traffic signs that were used as templates
->of the image manipulation processes
->of the brighness variations values used
->of the blending procedures


----------------------------------------------
Image format and naming
----------------------------------------------
The images created are of "jpg" format
with RGBA channels

   SIGN_X/XXX_YYY.jpg

The initial part (X) is used to distinguish the
sign class, while the remaining (XXX_YYY) firstly
indicated the sign in the file itself and the
example number.


----------------------------------------------
Additional information
----------------------------------------------

contact email: 
    
	asterga@essex.ac.uk


----------------------------------------------
Alexandros Stergiou
"The Driver's Assistant"

University of Essex,
School of Computer Science and
Electronic Engineering,
UK
----------------------------------------------
'''
text_file = open("SGTSD/Readme_Images.txt", "w")
text_file.write(content)
text_file.close()

In [ ]:
#List of paths for all SGTSD relevant files using exposure_manipulation
def create_paths_list(imgs_directory, bg_directory):
    directories = []
    for places in load_paths(imgs_directory): #List of places: either UK_rural or UK_urban
        for imgs in load_paths(places): #Folder for each bg image: eg. IMG_0
            dr = imgs.split('/')
            bg = bg_directory + '/' + dr[-2] + '/' + dr[-1] + ".png" #Retrieving relevant bg image
            for signs in load_paths(imgs): #Folder for each sign type: eg. SIGN_9
                for dmgs in load_paths(signs): #Folder for each damage type: eg. 0_HOLES
                    for png in load_paths(dmgs):
                        directories.append([png, bg])
    return directories #Directory for every single FILE and it's relevant bg FILE

In [ ]:
#List of paths for all SGTSD relevant files using fade_manipulation; backgrounds are assigned to 
def create_assigned_paths_list(imgs_directory, bg_directory): #TODO: is this the same as above?
    directories = []
    for places in load_paths(bg_directory): #Folder for each place: eg. UK_rural
        for imgs in load_paths(places): #Iterate through each b.g. image: eg. IMG_0
            for signs in load_paths(imgs_directory):  #Folder for each sign type: eg. SIGN_9
                for dmgs in load_paths(signs): #Folder for each damage type: eg. 9_HOLES
                    for png in load_paths(dmgs):
                        directories.append([png, imgs])
    return directories #Directory for every single FILE and it's relevant bg FILE

In [ ]:
if original is True:
    directories = create_paths_list("Traffic_Signs_Exposure_Manipulation","Google_search_backgrounds")
else:
    directories = create_assigned_paths_list("Traffic_Signs_Fade_Manipulation","Google_search_backgrounds")
print("Files to be generated: " + str(len(directories)))

In [ ]:
#Paths of images needed to generate examples for 'sign' with damage 'dmg'
def list_for_sign_x(sign, dmg, directories):
    l = []
    for elements in directories:
        foreground = elements[0].split('/')
        if (foreground[-2] == sign + dmg): #Eg. if (9_YELLOW == 4_ORIGINAL)
            l.append(elements)
    return l #Directory for every single sign and it's relevant background image

In [ ]:
final_directories = [] #Reformat list to have each sign and damage as their own dimensions
signs = load_paths('Traffic_Signs_Templates/1_Images')
dmgs = load_paths('Traffic_Signs_Templates/3_Damaged_Images')
for i in signs:
    head, tail = ntpath.split(i)
    sign, extension = tail.split('.') #Eg. sign == "9"

    sign_list = [] #List of damages, which are each list of signs
    for dmg in damage_types: #damage_types is from 'def damage_images:' cell
        sign_list.append(list_for_sign_x(sign, dmg, directories))
    final_directories.append(sign_list) #List of types -> lists of damages -> lists of signs

In [ ]:
#This generates all of the combined sign + background images (>1,000,000 originally)
direct = "SGTSD/Images"
folders = load_paths(direct)
n = [] #Numbers from the folder names for the signs
for folder in folders:
    head, tail = ntpath.split(folder)
    n.append(tail)
    
i = 0
for signs in final_directories: #Iterating through sign types
    print("Processed: " + str(float(i) / float(len(final_directories)) * 100) + " %")
    j = 0
    for damages in signs: #Iterating through damage types
        k = 0
        for dirs in damages: #dirs == the foreground and background images for one generated sign
            image = new_data(dirs[0], dirs[1]) #Combining background with exposure modified sign foreground
#             image.save(direct+"/"+n[i]+"/"+n[i]+"_"+str(j)+"/"+n[i]+"_"+str(j)+"_"+str(k)+".png")
            cv2.imwrite(direct+"/"+n[i]+"/"+n[i]+"_"+str(j)+"/"+n[i]+"_"+str(j)+"_"+str(k)+".png", image)
            k = k + 1
        j = j + 1
    i = i+1
print("Processed: "+str(100)+" %")

In [ ]:
string = '''
-------------------------------------
BREAKDOWN OF FILES GENERATED BY CLASS
-------------------------------------
'''
total = 0
for i in range(len(final_directories)):
    current = 0
    for j in range(len(final_directories[i])):
        current = current + len(final_directories[i][j])
    s = "Generated " + str(current) + " examples for sign class " + str(i + 1)
    string = string + '\n' + s + '\n'
    total = total + current
string = string + '\n' + "TOTAL: " + str(total) + '\n' + "Generated on: " + datetime.now().strftime("%Y-%m-%d %H:%M") + '\n'
string = string + "-------------------------------------"
text_file = open("SGTSD/generated_images_about.txt", "w")
text_file.write(string)
text_file.close()

In [ ]:
def png_to_jpeg(filepath):
    dirs = filepath.split('/')
    title,extension = dirs[-1].split('.')
    del dirs[-1]
    string = '/'.join(dirs)
    string = string + '/' + title + ".jpg"
    png = Image.open(filepath)
    png.load() # required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
    background.save(string, 'JPEG', quality=100)
    os.remove(filepath)

In [ ]:
dirs = direct = "SGTSD/Images"
i = 1
for path in load_paths(dirs):
    print("Processed: " + str(float(i - 1) / float(len(final_directories)) * 100) + " %")
    for image in load_paths(path):
        if (image.endswith("png")):
            png_to_jpeg(image)
    i = i + 1
print("Processed: " + str(100) + " %")

In [ ]:
shutil.rmtree("Traffic_Signs_Exposure_Manipulation")

In [ ]:
shutil.rmtree("Traffic_Signs_Fade_Manipulation")

In [ ]:
shutil.rmtree("Traffic_Signs_Templates/4_Transformed_Images")
shutil.rmtree("Traffic_Signs_Templates/3_Damaged_Images")
shutil.rmtree("Traffic_Signs_Templates/2_Processed_Images")

In [ ]:
shutil.rmtree("SGTSD") #Be careful with this one